### 롯데시네마

In [ ]:
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service #
from webdriver_manager.chrome import ChromeDriverManager # 
from selenium.webdriver.common.by import By


# 웹 브라우저와 연동을 위해
from selenium import webdriver
# Chrome 객체의 인자로 넣기 위해
from selenium.webdriver.chrome.service import Service
# 사용 중인 Chrome version과의 싱크를 맞추기 위해
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from datetime import date
import requests
import time

def getCinemaId():
    cinemaID_dict = {}

    with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
        driver.get("https://www.lottecinema.co.kr/NLCHS/Ticketing/Schedule")
        driver.implicitly_wait(10)
        
        cinemaID = []
        for i in range(1, 25): # (1~24) 서울 영화관 고유 아이디 추출
            element = driver.find_element(By.XPATH, '//*[@id="nav"]/ul/li[3]/div/ul/li[2]/div/ul/li[{}]/a'.format(i))
            
            thear_name = element.text  # href 속성 값을 가져옵니다.
            print(thear_name)
            # print(href)
            
            cinemaID.append(thear_name)  # href 값을 출력합니다.
            
            
        cinemaID_dict["서울"] = cinemaID
        # print(cinemaID_dict)
        
        cinemaID = []
        for i in range(1, 48): # (1~48) 경기/인천 영화관 고유 아이디 추출
            element = driver.find_element(By.XPATH, '//*[@id="nav"]/ul/li[3]/div/ul/li[3]/div/ul/li[{}]/a'.format(i))
            
            thear_name = element.text
            cinemaID.append(thear_name)
            
            cinemaID_dict["경기/인천"] = cinemaID

    return cinemaID_dict

cinemaID_dict = getCinemaId()

def getSigu(cinemaID_dict):
    # 
    # cinemaID_dict = getCinemaId() 
    result = []

    for city in cinemaID_dict:
        
        cinemaID_list = cinemaID_dict[city] # 0001
        print(cinemaID_list)
        for cinema_id in cinemaID_list:
            webdriver_options = webdriver.ChromeOptions()
            webdriver_options.add_argument('headless') # 화면 안보이기
            url = f'https://www.lottecinema.co.kr/NLCHS/Cinema/Detail?divisionCode=1&detailDivisionCode=1&cinemaID={cinema_id}'
    
            with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = webdriver_options) as driver:
                driver.get(url)
                driver.implicitly_wait(10)
                time.sleep(5)
                try:
                    # driver.send_keys(Keys.ESCAPE)
                    driver.find_element(By.CSS_SELECTOR, '#layerGetPopup > div.layer_header > button').click()
                    
                except:
                    try:
                        driver.find_element(By.CSS_SELECTOR, '#layerPopupMulti > li > div.layer_footer.ty2 > ul > li > button').click()
                    except:
                        pass

                address = driver.find_element(By.CSS_SELECTOR, '#contents > div.theater_top_wrap > div.info_wrap > dl.theater_info > dd.adr').text
            
                theater_name = driver.find_element(By.CSS_SELECTOR, '#contents > div.theater_top_wrap > div.theater_tit > h3').text
                print(address)
                sigu = address.strip().split(' ')[1]
                
    
                result.append(['롯데시네마']+ [theater_name]+ [city] + [sigu])
                print(['롯데시네마']+ [theater_name]+ [city] + [sigu])
    
    movie_total = pd.DataFrame(result,  columns=('theater_type',  'theater_name', 'location', 'sigu'))
    movie_total.to_csv(f'롯데시네마 극장정보_{datetime.datetime.now()}.csv', encoding='utf-8', index = False )
            
    #contents > div.theater_top_wrap > div.info_wrap > dl.theater_info > dd.adr

getSigu(cinemaID_dict)

### CGV

In [ ]:
import pandas as pd
import json

def crawling_title_starttime(areacode, theaterCode, theaterName, json, driver):
    if areacode=="01":
        location="서울"
    elif areacode=="02":
        location="경기도"
    elif areacode=="202":
        location="인천"

    try:
        driver.get(f"http://www.cgv.co.kr/theaters/?areacode={areacode}&theaterCode={theaterCode}&date=20230502")
        driver.implicitly_wait(10)
        address = driver.find_element(By.CSS_SELECTOR, '#contents > div.wrap-theater > div.sect-theater > div > div.box-contents > div.theater-info > strong').text
        print(address)
        sigu = address.strip().split(' ')[1]
        
        
        json.append({"theater_type": "CGV", "theater_name": theaterName, "location": location, "sigu": sigu })
        print(json)        
        return json
    except Exception as error:
        print(error)
        return json


def crawling_location_theatername():
    soup=BeautifulSoup(requests.get("http://www.cgv.co.kr/theaters/").text, "html.parser")
    script=soup.find("div", id="contents").script.string
    expression = r'"RegionCode":\s*"([^"]*)"\s*,\s*"TheaterCode":\s*"([^"]*)"\s*,\s*"TheaterName":\s*"([^"]*)'
    matches = re.findall(expression, script, re.DOTALL)
    movie_json = []
    driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    for match in matches:
        if match[0] in ["01", "02", "202"]:
            movie_json=crawling_title_starttime(match[0], match[1], match[2], movie_json, driver)
            time.sleep(0.5)
        else:
            break
    driver.quit()

    print(movie_json)
    with open('CGV.json', 'w', encoding='utf-8') as f:
        json.dump(movie_json, f, ensure_ascii=False)

    return movie_json


if __name__=="__main__":
    crawling_location_theatername()

### 메가박스

In [ ]:
import pandas as pd
import datetime
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service #
from webdriver_manager.chrome import ChromeDriverManager # 
from selenium.webdriver.common.by import By



def theater_crawl(result, theater_name, theater_name_url):
    '''영화관 지점별 크롤링'''
    
    webdriver_options = webdriver.ChromeOptions()
    webdriver_options.add_argument('headless') # 화면 안보이기
    driver_detail = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = webdriver_options) # 
    # driver_detail = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver_options )
    driver_detail.get(theater_name_url)
    driver_detail.implicitly_wait(2)

    # 주소
    address = driver_detail.find_element(By.CSS_SELECTOR, '#tab01 > ul:nth-child(9) > li').text
    print(address)
    driver_detail.implicitly_wait(2)
    sigu = address.strip().split(' ')[3]


    
    result.append(['MEGABOX']+ [theater_name]+ [''] + [sigu])
    print(['MEGABOX']+ [theater_name]+ [''] + [sigu])
    
    driver_detail.quit()

    return result
    


def location_crawl(driver, location_element):
    '''지역별 크롤링'''
    result = []
    location_element.click()
    theater_name_elements = driver.find_elements(By.CSS_SELECTOR, 'div.theater-place > ul > li.on > div > ul > li > a')


    for theater_name_element in theater_name_elements:
        # 테스트
        
        theater_name_url = theater_name_element.get_attribute('href')
        theater_name = theater_name_element.text
        
        # 지점별 상영시간 크롤링 
        result = theater_crawl(result, theater_name, theater_name_url)
        print(f'{theater_name} 완료')
    
    print(result)
    movie_tbl = pd.DataFrame(result,  columns=('theater_type',  'theater_name', 'location', 'sigu'))
    return movie_tbl
        
def megabox_crawl():
    print('>>> 크롤링 시작')
    main_url = 'https://www.megabox.co.kr/theater/list'
    webdriver_options = webdriver.ChromeOptions()
    # webdriver_options.add_argument('headless') # 화면 안보이기

    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # 
    driver.get(main_url)

    # 서울, 경기, 인천
    location_elements = driver.find_elements(By.CSS_SELECTOR, 'li > button.sel-city')
    # movie_total = pd.DataFrame(columns=('theater_type'  'theater_name' 'location' 'movie_title' 'start_time' 'run_time'))
    # 앞 3개만 크롤링 (서울, 경기, 인천) / 0, 1, 2
    location = {0:'서울', 1:'경기/인천', 2:'경기/인천'}
    for i in range(3):
        # # 테스트 
        # if i ==1 :
        #     break

        print(f'>>> {location[i]} 시작')
        movie_tbl = location_crawl(driver, location_elements[i])
        movie_tbl['location'] = location[i]
        print(f'>>> {location[i]} 완료')
    
        try:
            movie_total = pd.concat([movie_total, movie_tbl])
        except:
            movie_total = movie_tbl

    WORKING_DIR = os.getcwd()
    movie_total.to_csv(f'메가박스 극장정보_{datetime.datetime.now()}.csv', encoding='cp949', index = False )
    # movie_total.to_csv(os.path.join(WORKING_DIR, f'output_file/메가박스 상영시간_{datetime.datetime.now()}.csv'), encoding='utf-8', index = False )
    
    driver.quit()


if __name__ == '__main__':
    megabox_crawl()
    